In [1]:
# Dependencies
import pandas as pd
import requests
import json
import gmaps
# Google developer API key
from config import gkey

In [2]:
#Zillow data
zillow_data1= "data_sources/zillow_data/zillow_medianhome_matched.csv"
zillow_pd= pd.read_csv(zillow_data1)
zillow_pd.head()

,Zip,City,County,Type,Area,Value,Year
0,76574,Taylor,Williamson County,Rural,North,106847.78,2011
1,76574,Taylor,Williamson County,Rural,North,106776.56,2012
2,76574,Taylor,Williamson County,Rural,North,107119.61,2013
3,76574,Taylor,Williamson County,Rural,North,107849.07,2014
4,76574,Taylor,Williamson County,Rural,North,109096.49,2015


In [3]:
#Census data
census_data1= "data_sources/census_data/combined_census_cleaned.csv"
census_pd= pd.read_csv(census_data1)
census_pd.head()

,Zip,Total-Population,Hispanic,White,Black,Native,Asian,Other,Total,Diversity Index,Year,City
0,76574,17213,35.3,54.0,8.6,0.0,1.8,0.3,100.0,0.576062,2011,Taylor
1,76574,17775,34.5,53.6,8.9,0.0,2.0,1.0,100.0,0.585258,2012,Taylor
2,76574,18037,36.8,51.1,9.8,0.0,0.2,2.2,100.1,0.593363,2013,Taylor
3,76574,18135,35.3,51.9,9.9,0.0,0.4,2.5,100.0,0.595588,2014,Taylor
4,76574,18281,41.4,46.0,9.6,0.1,0.7,2.1,99.9,0.607297,2015,Taylor


In [4]:
#Merged on zip and year
merge_data= pd.merge(zillow_pd,census_pd, on=["Zip","Year"])
merge_data = merge_data.drop(["City_y"], axis=1)
merge_data= merge_data.rename(columns={"City_x":"City"})
merge_data.head(3)

,Zip,City,County,Type,Area,Value,Year,Total-Population,Hispanic,White,Black,Native,Asian,Other,Total,Diversity Index
0,76574,Taylor,Williamson County,Rural,North,106847.78,2011,17213,35.3,54.0,8.6,0.0,1.8,0.3,100.0,0.576062
1,76574,Taylor,Williamson County,Rural,North,106776.56,2012,17775,34.5,53.6,8.9,0.0,2.0,1.0,100.0,0.585258
2,76574,Taylor,Williamson County,Rural,North,107119.61,2013,18037,36.8,51.1,9.8,0.0,0.2,2.2,100.1,0.593363


In [5]:
#Filter used data by 2017
merge_data_n=merge_data[merge_data["Year"]==2017]
merge_data_n=merge_data.groupby(["City"]).mean()
merge_data_n["city"]=merge_data_n.index
merge_data_n["Diversity Index"]=merge_data_n["Diversity Index"]*100

In [6]:
# latitude and longitude
merge_data_n["Lat"]=0.001
merge_data_n["Lng"]=0.001
for  row in merge_data_n["city"]:
    target_city=row
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city+", Texas", gkey)
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
    
    # Print the json
    print(geo_data)
   
    # Extract latitude and longitude
    lat = geo_data["results"][0]["geometry"]["location"]["lat"]
    lng = geo_data["results"][0]["geometry"]["location"]["lng"]
    merge_data_n["Lat"][row]=lat
    merge_data_n["Lng"][row]=lng
#     Print the latitude and longitude
    print('''
        City: {0}
        Latitude: {1}
        Longitude: {2}
        '''.format(target_city, lat, lng))

{'results': [{'address_components': [{'long_name': 'Austin', 'short_name': 'Austin', 'types': ['locality', 'political']}, {'long_name': 'Travis County', 'short_name': 'Travis County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Texas', 'short_name': 'TX', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Austin, TX, USA', 'geometry': {'bounds': {'northeast': {'lat': 30.5168629, 'lng': -97.5684199}, 'southwest': {'lat': 30.0986589, 'lng': -97.9383829}}, 'location': {'lat': 30.267153, 'lng': -97.7430608}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 30.5168629, 'lng': -97.5684199}, 'southwest': {'lat': 30.0986589, 'lng': -97.9383829}}}, 'place_id': 'ChIJLwPMoJm1RIYRetVp1EtGm10', 'types': ['locality', 'political']}], 'status': 'OK'}

        City: Austin
        Latitude: 30.267153
        Longitude: -97.7430608
        


/anaconda3/envs/BH6Project1/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/BH6Project1/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'results': [{'address_components': [{'long_name': 'Bastrop', 'short_name': 'Bastrop', 'types': ['locality', 'political']}, {'long_name': 'Bastrop County', 'short_name': 'Bastrop County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Texas', 'short_name': 'TX', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '78602', 'short_name': '78602', 'types': ['postal_code']}], 'formatted_address': 'Bastrop, TX 78602, USA', 'geometry': {'bounds': {'northeast': {'lat': 30.1377119, 'lng': -97.2775851}, 'southwest': {'lat': 30.0883561, 'lng': -97.36482490000002}}, 'location': {'lat': 30.1104947, 'lng': -97.3152701}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 30.1377119, 'lng': -97.2775851}, 'southwest': {'lat': 30.0883561, 'lng': -97.36482490000002}}}, 'place_id': 'ChIJadvR6FuaRIYRYNmtW2Y8DTQ', 'types': ['locality', 'political']}], 'status': 'OK

{'results': [{'address_components': [{'long_name': 'Georgetown', 'short_name': 'Georgetown', 'types': ['locality', 'political']}, {'long_name': 'Williamson County', 'short_name': 'Williamson County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Texas', 'short_name': 'TX', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Georgetown, TX, USA', 'geometry': {'bounds': {'northeast': {'lat': 30.742566, 'lng': -97.585174}, 'southwest': {'lat': 30.570766, 'lng': -97.832725}}, 'location': {'lat': 30.6332618, 'lng': -97.6779842}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 30.742566, 'lng': -97.585174}, 'southwest': {'lat': 30.570766, 'lng': -97.832725}}}, 'place_id': 'ChIJ3_vaVP3VRIYRCTrAlN407GE', 'types': ['locality', 'political']}], 'status': 'OK'}

        City: Georgetown
        Latitude: 30.6332618
        Longitude: -97.67798

{'results': [{'address_components': [{'long_name': 'Luling', 'short_name': 'Luling', 'types': ['locality', 'political']}, {'long_name': 'Caldwell County', 'short_name': 'Caldwell County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Texas', 'short_name': 'TX', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '78648', 'short_name': '78648', 'types': ['postal_code']}], 'formatted_address': 'Luling, TX 78648, USA', 'geometry': {'bounds': {'northeast': {'lat': 29.7048079, 'lng': -97.58989389999999}, 'southwest': {'lat': 29.64521, 'lng': -97.6724079}}, 'location': {'lat': 29.68050659999999, 'lng': -97.647498}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 29.7048079, 'lng': -97.58989389999999}, 'southwest': {'lat': 29.64521, 'lng': -97.6724079}}}, 'place_id': 'ChIJNWByMiJPQ4YRr3UhGYDLL-Q', 'types': ['locality', 'political']}], 'status': 'O

{'results': [{'address_components': [{'long_name': 'Smithville', 'short_name': 'Smithville', 'types': ['locality', 'political']}, {'long_name': 'Bastrop County', 'short_name': 'Bastrop County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Texas', 'short_name': 'TX', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}, {'long_name': '78957', 'short_name': '78957', 'types': ['postal_code']}], 'formatted_address': 'Smithville, TX 78957, USA', 'geometry': {'bounds': {'northeast': {'lat': 30.040103, 'lng': -97.1198769}, 'southwest': {'lat': 29.991439, 'lng': -97.177088}}, 'location': {'lat': 30.0085542, 'lng': -97.15943209999999}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 30.040103, 'lng': -97.1198769}, 'southwest': {'lat': 29.991439, 'lng': -97.177088}}}, 'place_id': 'ChIJq08GvwiBRIYRKLmxKeQqtd0', 'types': ['locality', 'political']}], 'status': 'OK'}



 For the heat map, I used the merged data from Zillow and Census .I filtered merged data by 2017 to understand how diverse is Austin and surrounding areas.I added latitude and longitude to my data chart as well to use heatmap. In short, Round Rock, Pflugerville, Leander, Buda, Mountain City, San Marcos and part of the Cedar Park have diverse population compare to rest of the Austin and surrounding areas.

In [7]:
# Plot Heatmap
weight=locations = merge_data_n["Diversity Index"].astype(float)
locations = merge_data_n[["Lat", "Lng"]].astype(float)
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, max_intensity=80, weights=weight,
                                 point_radius=0.15)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))